### Resources for finetuning:
https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb
 
https://github.com/huggingface/notebooks/blob/main/examples/question_answering.ipynb

https://github.com/huggingface/transformers/tree/main/notebooks

Paper:
https://arxiv.org/pdf/2205.01068.pdf

In [2]:
from huggingface_hub import notebook_login

#notebook_login()
!huggingface-cli login


'huggingface-cli' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
from datasets import load_dataset

dataset = load_dataset("pile-of-law/pile-of-law",'r_legaladvice')

C:\Users\thoma\anaconda3\envs\hf\lib\site-packages\huggingface_hub-0.10.1-py3.8.egg\huggingface_hub\utils\_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
Found cached dataset pile-of-law (C:/Users/thoma/.cache/huggingface/datasets/pile-of-law___pile-of-law/r_legaladvice/0.0.0/acacf3e29a952ba9026148b979cb438151ebd33f842f5779a213967033c88619)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(dataset["train"])


,text,created_timestamp,downloaded_timestamp,url
0,"Title: [TX] Are you liable if someone gets bitten by a wild snake on your property?\nQuestion:[deleted]\nAnswer #1: Rob should let his home owners insurance handle itAnswer #2: Texas law is pretty friendly to landowners in these cases. Specifically see 75.002.\nhttp://www.statutes.legis.state.tx.us/Docs/CP/htm/CP.75.htm\n\nIf Mary's family decided not to take her to the closest hospital, then maybe she should sue them. Sheesh.Answer #3: Does Mary have dementia? Because it doesn't sound like she was in her right mind. If so, who is in charge of her safety? Answer #4: This sounds like something that happened in like half of my Oregon Trail playthroughs.Answer #5: Like an update on this if anything else happens. Interesting. And I don't think it's a crazy idea for this to be an elaborate insurance scam. People will do some extreme things for the sake of greed. It's hard for normal people to wrap there head around that kind of insanity but it's very real.Answer #6: &gt;They didn't put a tourniquet on it\n\nI don't think a tourniquet is appropriate in this situation. They should only be used if there is a chance of the person bleeding to death. Instead, the effected limb should be kept below the heart as much as possible and the person should remain calm to keep their heart from racing and distributing the venom quicker. Then get medical attention as soon as possible.",05-29-2016,11-09-2021,https://www.reddit.com/r/legaladvice/comments/4ll0mu/tx_are_you_liable_if_someone_gets_bitten_by_a/
1,"Title: My father hit a vehicle(s) while driving and drove off without notifying anyone. The police became involved and he wants me to take the fall for it.\nQuestion:This happened very late at night and shortly after the incident the police were called and through witnesses statements they found our address and visited our home, after some questioning they left but told us that they would be back later after an unsuccessful attempt at getting an answer from my father about who was driving the vehicle at the time of the incident. My father wants me to take the fall for it as he believes I'll receive a lighter punishment than he would as I do not have a full license and only have a learner's license. I am 20 years old and thus an adult and not a minor. This is also in Australia.\n\nWhat should I do?\nAnswer #1: &gt; What should I do?\n\nSay no.",11-09-2019,09-25-2021,https://www.reddit.com/r/legaladvice/comments/dtznsx/my_father_hit_a_vehicles_while_driving_and_drove/
2,"Title: Asking my college to change my curriculum for a mental disability?\nQuestion:Hi everyone. I feel kind of stuck here. \n\n\n I am a cello performance major at a college in Colorado. I've also grown up with a number of mental problems including a diagnosis of Tourette Syndrome, which caused a few other problems like bullying and low self esteem indirectly.\n\n\nLast year, I played an awesome Junior recital and now I have to play a Senior recital this year. \n\nThe problem is, the junior recital kind of killed my brain. After it was all over and the stress was gone, my tourettes were permanently worse. I also think I might have developed a mild case of insomnia since then, on top of being plagued by suicidal thoughts and depression leading up to the recital and after. I'm worried about what doing a Senior recital, (which is twice as long and grueling to prepare and play) will do to me mentally. I've already started seeing long-term mental health effects that have yet to go away. One of the worst tics I've developed is related to the muscles in my bowing arm.\n\n\nDo I have anything to stand on if I want my college to forgo the Senior recital requirement? If I got a doctor's note, who should I bring it to to make this change?\n\n My alternate suggestions would be something like a Senior Youtube channel, or other work with the instrument that my cello professor would deem suitable, but would be more doable for someone in my po

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")
model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m")

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [8]:
dataset["train"][161]

{'text': "Title: Personal information and walmart\nQuestion:This is whats going on. I bought a nerf gun for my son at Wal-Mart for Christmas. I found the same nerf gun $40 cheaper, so naturally returned it to Wal-Mart. I got a call yesterday (well voicemail) from a Wal-Mart customer who bought the nerf gun.. it gets weird here.. they said Wal-Mart put my personal information on the box and decided to use it to contact me to see if there is anything wrong with it. I am naturally paranoid with my personal information as is most people. Why would they do this? What should I do? sorry if I am in the wrong sub-reddit.. Thanks in advance \n\np.s. I do not frequent Wal-Mart often, it was the only place for this particular nerf gun at the moment \n\nI am in the chicago area \nAnswer #1: Your information is personal in nature, but is by no means private.  Your full name, birth date, home address, and sometimes phone numbers are all generally a matter of public record and readily available to an

In [9]:
tokenize_function(dataset["train"][161])

{'input_ids': [2, 46525, 35, 13129, 335, 8, 21788, 15964, 50118, 45641, 35, 713, 16, 45676, 164, 15, 4, 38, 2162, 10, 38286, 506, 1751, 13, 127, 979, 23, 6092, 12, 13012, 13, 1619, 4, 38, 303, 5, 276, 38286, 506, 1751, 68, 1749, 7246, 6, 98, 8366, 1835, 24, 7, 6092, 12, 13012, 4, 38, 300, 10, 486, 2350, 36, 3056, 30118, 10555, 43, 31, 10, 6092, 12, 13012, 2111, 54, 2162, 5, 38286, 506, 1751, 7586, 24, 1516, 7735, 259, 7586, 51, 26, 6092, 12, 13012, 342, 127, 1081, 335, 15, 5, 2233, 8, 1276, 7, 304, 24, 7, 1511, 162, 7, 192, 114, 89, 16, 932, 1593, 19, 24, 4, 38, 524, 8366, 33554, 19, 127, 1081, 335, 25, 16, 144, 82, 4, 2612, 74, 51, 109, 42, 116, 653, 197, 38, 109, 116, 6661, 114, 38, 524, 11, 5, 1593, 2849, 12, 48724, 7586, 4557, 11, 3316, 1437, 50118, 50118, 642, 4, 29, 4, 38, 109, 45, 7690, 6092, 12, 13012, 747, 6, 24, 21, 5, 129, 317, 13, 42, 1989, 38286, 506, 1751, 23, 5, 1151, 1437, 50118, 50118, 100, 524, 11, 5, 1855, 42938, 443, 1437, 50118, 33683, 849, 134, 35, 2486, 335, 16, 

In [10]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=1, remove_columns=["text","created_timestamp","downloaded_timestamp","url"])

Loading cached processed dataset at C:/Users/thoma/.cache/huggingface/datasets/pile-of-law___pile-of-law/r_legaladvice/0.0.0/acacf3e29a952ba9026148b979cb438151ebd33f842f5779a213967033c88619\cache-3bcde4656ba684b1.arrow
Loading cached processed dataset at C:/Users/thoma/.cache/huggingface/datasets/pile-of-law___pile-of-law/r_legaladvice/0.0.0/acacf3e29a952ba9026148b979cb438151ebd33f842f5779a213967033c88619\cache-d3ffc92b6cd8bfa1.arrow


In [11]:
tokenized_dataset["train"][1]

{'input_ids': [2,
  46525,
  35,
  38,
  206,
  951,
  16,
  3433,
  88,
  127,
  512,
  7,
  120,
  13543,
  13,
  162,
  145,
  10,
  1099,
  1393,
  411,
  377,
  536,
  50118,
  45641,
  35,
  713,
  16,
  11,
  2042,
  4,
  3727,
  11,
  392,
  6,
  38,
  13636,
  4024,
  5,
  1593,
  169,
  159,
  10,
  5342,
  3610,
  65,
  169,
  921,
  142,
  38,
  56,
  45,
  551,
  127,
  33286,
  5679,
  29,
  14,
  183,
  4,
  17397,
  117,
  1677,
  58,
  567,
  1567,
  162,
  8,
  25,
  1010,
  25,
  38,
  5426,
  24,
  6,
  38,
  4024,
  11,
  5,
  4806,
  6625,
  13,
  5,
  1079,
  9,
  5,
  1803,
  454,
  38,
  115,
  1004,
  4,
  38,
  21,
  2198,
  7619,
  8435,
  66,
  59,
  24,
  6,
  98,
  38,
  1276,
  7,
  213,
  124,
  184,
  8,
  38003,
  2185,
  4,
  38,
  1299,
  3668,
  6587,
  59,
  24,
  4,
  287,
  38,
  21,
  1428,
  184,
  6,
  38,
  5324,
  10,
  512,
  511,
  162,
  4,
  38,
  554,
  7,
  185,
  124,
  3197,
  8,
  1076,
  21242,
  7,
  860,
  7,
  2217,
  123,
  13

In [12]:
#block_size = tokenizer.model_max_length
block_size = 256

In [13]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [14]:
lm_datasets = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=1,
)

Loading cached processed dataset at C:/Users/thoma/.cache/huggingface/datasets/pile-of-law___pile-of-law/r_legaladvice/0.0.0/acacf3e29a952ba9026148b979cb438151ebd33f842f5779a213967033c88619\cache-2f4e3edc68f1dc89.arrow
Loading cached processed dataset at C:/Users/thoma/.cache/huggingface/datasets/pile-of-law___pile-of-law/r_legaladvice/0.0.0/acacf3e29a952ba9026148b979cb438151ebd33f842f5779a213967033c88619\cache-5cf3a0e4382b5555.arrow


In [12]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    f"opt125m-finetuned-pileoflaw_reddit",
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,
    save_strategy='epoch',
    per_device_train_batch_size=12 
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

Using cuda_amp half precision backend


In [13]:
trainer.train()

C:\Users\thoma\anaconda3\envs\hf\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 202713
  Num Epochs = 3
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 50679


Step,Training Loss
500,3.161900
1000,3.105100
1500,3.097400
2000,3.078400
2500,3.072100
3000,3.063100
3500,3.052000
4000,3.046700
4500,3.043200
5000,3.031200


Saving model checkpoint to opt125m-finetuned-pileoflaw_reddit\checkpoint-16893
Configuration saved in opt125m-finetuned-pileoflaw_reddit\checkpoint-16893\config.json
Model weights saved in opt125m-finetuned-pileoflaw_reddit\checkpoint-16893\pytorch_model.bin
Saving model checkpoint to opt125m-finetuned-pileoflaw_reddit\checkpoint-33786
Configuration saved in opt125m-finetuned-pileoflaw_reddit\checkpoint-33786\config.json
Model weights saved in opt125m-finetuned-pileoflaw_reddit\checkpoint-33786\pytorch_model.bin
Saving model checkpoint to opt125m-finetuned-pileoflaw_reddit\checkpoint-50679
Configuration saved in opt125m-finetuned-pileoflaw_reddit\checkpoint-50679\config.json
Model weights saved in opt125m-finetuned-pileoflaw_reddit\checkpoint-50679\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=50679, training_loss=2.923679765575477, metrics={'train_runtime': 19596.794, 'train_samples_per_second': 31.033, 'train_steps_per_second': 2.586, 'total_flos': 7.9450937524224e+16, 'train_loss': 2.923679765575477, 'epoch': 3.0})

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [15]:
model = AutoModelForCausalLM.from_pretrained("opt125m-finetuned-pileoflaw_reddit/checkpoint-50679")

In [24]:
from transformers import pipeline
generator = pipeline('text-generation',model=model,tokenizer=tokenizer, do_sample=True)

In [30]:
generator("\nQuestion: My employer fired me today. Can I still receive severance?  \nAnswer #1:",max_length=256)

[{'generated_text': "\nQuestion: My employer fired me today. Can I still receive severance?  \nAnswer #1: Yes and no.  They fired you for not getting paid when you work more than 50 hours a week.  If that was true then they'd have been fired for missing too many paychecks or just working too many hours."}]

In [17]:
model.push_to_hub('bettercallopt/opt_125m_legaladvice')


ValueError: Token is required (write-access action) but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `notebook_login`. See https://huggingface.co/settings/tokens.